In [ ]:
import polars as pl

df_inad = pl.read_excel('.sample/Bandeira3/Inadimplencia Condomínio.xlsx')

if "Bloco" not in df_inad.columns:
    df_inad = df_inad.with_columns([
        pl.lit(1).alias("Bloco")
    ])
df_inad = df_inad.select([
    pl.col('Bloco').cast(pl.Int64),
    pl.col('Unidade').cast(pl.Int64),
    pl.col('Vlr Total'),
])

df_inad = df_inad.with_columns([
    pl.col('Vlr Total')
        .str.replace(",", ".")
        .cast(pl.Float64)
])

df_inad = df_inad.group_by(
    pl.col('Bloco'),
    pl.col('Unidade'),
    maintain_order=True,
).agg(
    pl.col('Vlr Total').sum()
)

print(df_inad)

In [ ]:
#Possivel melhoria: Verificar se existe telefone na coluna Telefone2 e preencher conforme null Telefone1

df_unit_data = pl.read_excel('/home/notificaTu/.sample/Bandeira3/Unidades.xlsx')

df_unit_data = df_unit_data.select([
    pl.col('Unidade'),
    pl.col('ProprietarioCpfCnpj').cast(str).str.replace(',', '').alias('CPF/CNPJ'),
    pl.col('ProprietarioNome').alias('Proprietario'),
    pl.col('ProprietarioTelefone1').cast(str).alias('Telefone'),
    pl.col('ProprietarioEmail1').alias('E-mail'),
])

df_unit_data = df_unit_data.with_columns([
    pl.when(
        pl.col('Telefone').is_not_null() & 
        ~pl.col('Telefone').str.starts_with('67')
    )
    .then(pl.lit('+5567') + pl.col('Telefone'))
    .otherwise(pl.lit('+55') + pl.col('Telefone'))
    .alias('Telefone')
])

print(df_unit_data)

In [36]:
df_final = df_inad.join(df_unit_data, on='Unidade')

df_final.write_excel('.sample/Bandeira3/Inadimplencia_Final.xlsx')